In [1]:
import os

In [2]:
%pwd

'd:\\Ahmed ali\\AI Project\\Text-Summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Ahmed ali\\AI Project\\Text-Summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

[2024-12-18 20:04:00,707: INFO: __init__: Logging setup is complete.]
[2024-12-18 20:04:00,707: ERROR: __init__: This is an example of an error log.]


In [31]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        self.config.artifacts_root = Path(self.config.artifacts_root)  # Explicit conversion
        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # Convert all paths to Path objects
        root_dir = Path(config.root_dir)
        local_data_file = Path(config.local_data_file)
        unzip_dir = Path(config.unzip_dir)

        # Create necessary directories
        create_directories([root_dir])

        return DataIngestionConfig(
            root_dir=root_dir,
            source_URL=config.source_URL,
            local_data_file=local_data_file,
            unzip_dir=unzip_dir,
        )


In [32]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [33]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [34]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-18 21:06:29,395: INFO: common: YAML file 'config\config.yaml' loaded successfully]
[2024-12-18 21:06:29,398: INFO: common: YAML file 'params.yaml' loaded successfully]
[2024-12-18 21:06:29,400: INFO: common: Directory created: artifacts]
[2024-12-18 21:06:29,403: INFO: common: Directory created: artifacts\data_ingestion]
[2024-12-18 21:09:04,337: INFO: 1434958058: artifacts\data_ingestion\data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4794:4CB2:CBAA72:D13A05:67631D37
Accept-Ranges: bytes
Date: Wed, 18 Dec 2024 19:06:31 GMT
Via: 1.1 varnish
X-Served-By: cache-mrs10551-MRS
X-Cache: MISS
X-